In [1]:
!pip install py2neo

In [2]:
!pip install neo4j

In [3]:
from py2neo import Graph,Node,Relationship
from neo4j import GraphDatabase

In [4]:
graph = Graph('neo4j@bolt://localhost:7689', auth=('neo4j', 'Cm@ck6969'))

ConnectionUnavailable: Connection has been closed

In [ ]:
cypher = 'LOAD CSV WITH HEADERS FROM "file:///SpeedDatingData.csv" AS row \
WITH row WHERE NOT row.age IS null AND NOT row.iid IS null AND NOT row.pid IS null AND NOT row.age_o IS null and NOT row.race IS null and NOT row.race_o IS null and NOT row.match IS null and NOT row.int_corr IS null and NOT row.gender is null AND NOT row.samerace IS null AND NOT row.age_o IS null \
MERGE(p1 :Person {id:row.iid,age:toInteger(row.age), race:toInteger(row.race) }) MERGE( p2: Person {id:row.pid, age:toInteger(row.age_o), race:toInteger(row.race_o)}) MERGE((p1) - [:Date {match: toInteger(row.match), int_corr: row.int_corr, race_diff:toInteger(row.samerace), age_diff:abs(toInteger(row.age)- toInteger(row.age_o))}] -> (p2)) SET p1.gender = toInteger(row.gender)'

In [ ]:
result = graph.run(cypher)
y = 'MATCH(n:Person) RETURN COUNT(n)'


In [ ]:
q = graph.query(y)
q

In [ ]:
males_cypher = 'MATCH(n:Person) WHERE n.gender = 0 RETURN COUNT(n)'
males = graph.query(males_cypher)
males

In [ ]:
females_cypher = 'MATCH(n:Person) WHERE n.gender = 1 RETURN COUNT(n)'
females = graph.query(females_cypher)
females

In [ ]:
num_match_cypher = 'match (dater:Person) -[date:Date]->(datee:Person)-[date1:Date]->(dater) \
where date.match=1 and date1.match =1 return count(date)/2'
num_matches = graph.query(num_match_cypher)
num_matches

In [ ]:
delete = 'MATCH (n) DETACH DELETE n'
graph.run(delete)

In [ ]:
cypher = 'LOAD CSV WITH HEADERS FROM "file:///SpeedDatingData.csv" AS row \
WITH row WHERE NOT row.age IS null AND NOT row.iid IS null AND NOT row.pid \
IS null AND NOT row.age_o IS null and NOT row.race IS null and NOT row.race_o IS null \
and NOT row.match IS null and NOT row.int_corr IS null and NOT row.gender is null AND NOT \
row.samerace IS null AND NOT row.age_o IS null and not row.intel_o is null  and not row.intel is \
null and not row.income is null \
MERGE(p1 :Person {id:row.iid,age:toInteger(row.age), \
race:toInteger(row.race),intelligence:toInteger(row.intel),income:toInteger(row.income) }) \
MERGE( p2: Person {id:row.pid, age:toInteger(row.age_o), race:toInteger(row.race_o),intelligence:toInteger(row.intel_o)}) \
MERGE((p1) - [:Date {match: toInteger(row.match), \
int_corr: row.int_corr, race_diff:toInteger(row.samerace), \
age_diff:abs(toInteger(row.age)- toInteger(row.age_o)),intel_diff:abs(toInteger(row.intel)- toInteger(row.intel_o))}] -> \
(p2)) SET p1.gender = toInteger(row.gender)'
graph.run(cypher)

In [ ]:
query = 'match (dater:Person) -[date:Date]->(datee:Person) \
return max(dater.age),min(dater.age),avg(dater.age),stDev(dater.age), \
max(date.age_diff),avg(dater.income),stDev(dater.income),min(date.age_diff), \
avg(date.age_diff), stDev(date.age_diff), avg(date.race_diff),avg(date.intel_diff), \
stDev(date.intel_diff), avg(abs(dater.income-datee.income)),stDev(abs(dater.income-datee.income)),\
max(dater.income),min(dater.income),max(abs(dater.income-datee.income)),min(abs(dater.income-datee.income)),\
max(date.intel_diff),avg(dater.intelligence),stDev(dater.intelligence)'
graph.query(query)

In [ ]:
cypher = 'match(dater:Person) \
return max(dater.age),max(dater.income),max(dater.intelligence)'
graph.query(cypher)

In [ ]:
cypher = 'match (dater:Person) -[date:Date]->(datee:Person) \
return max(dater.age),min(dater.age),avg(dater.age),stDev(dater.age), \
max(date.age_diff),avg(dater.income),stDev(dater.income),min(date.age_diff), avg(date.age_diff), \
stDev(date.age_diff), avg(date.race_diff),avg(date.intel_diff), stDev(date.intel_diff), \
avg(abs(dater.income-datee.income)),stDev(abs(dater.income-datee.income)),max(dater.income),min(dater.income),\
max(abs(dater.income-datee.income)),min(abs(dater.income-datee.income)),max(date.intel_diff),avg(dater.intelligence),\
stDev(dater.intelligence),avg(abs(dater.gender-datee.gender))'
graph.query(cypher)

In [ ]:
homosexual_cypher = 'match (dater:Person) -[date:Date]->(datee:Person) where dater.gender = datee.gender return count(date)'
graph.query(homosexual_cypher)

In [ ]:
visualization_cypher = 'CALL db.schema.visualization()'
graph.query(visualization_cypher)
#Is this what was wanted for Q1?

Choices for columns:  'age', 'income'.  We can see that the average age difference and stDev are slightly lower for matches than for dates.  However, most of the dataset ages are 22-30, so this dataset may be biased.  We also see that the average income difference in a match is about the standard deviation of income in the data set.  I think race is irrelevant because the dataset shows that the average match is .433, which is about a coin toss for same vs different race.

In [ ]:
import pandas as pd
df = pd.read_csv("SpeedDatingData.csv")
df

In [ ]:
list(df)

In [ ]:
df2 = pd.read_csv("Speeddatingreduced.csv")
df2

In [ ]:
list(df2)

In [ ]:
df2['match'].unique()

In [ ]:
d = {x:[] for x in df2['dater'].unique()}
d

In [ ]:
for i in range(len(df2)):
    if df2.iloc[i]['match']==1:
        d[df2.iloc[i]['dater']].append(int(df2.iloc[i]['datee']))

In [ ]:
d

In [ ]:
count = 0#checking work
for k in d.keys():
    for p in d[k]:
        if k in d[p]:
            count+=1
            
print(count/2)

In [ ]:
df3 = df.copy()
for col in list(df3):
    if not col in ['iid','gender','pid','match','income','age','age_o']:
        df3.drop(col,inplace=True,axis=1)
df3.dropna(inplace=True)

In [ ]:
df3

In [ ]:
income_o = {}
for i in range(len(df)):
    income_o[int(df.iloc[i]['iid'])] = df.iloc[i]['income']
income_o

In [ ]:
incomes = []
for i in range(len(df3)):
    incomes.append(income_o[int(df3.iloc[i]['pid'])])
incomes

In [ ]:
inc = pd.DataFrame(incomes,columns=['income_o'])
inc

In [ ]:
df3 = pd.concat(objs=[df3,inc],axis=1)

In [ ]:
df3


In [ ]:
df3.dropna(inplace=True)

In [ ]:
df3

In [ ]:
y = df3['match']
y

In [ ]:
df3.drop('match',axis=1,inplace=True)

df3

In [ ]:
df3.drop('iid',axis=1,inplace=True)
df3.drop('pid',axis=1,inplace=True)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df3, y, test_size=0.4, random_state=0)
s = SVC()
s.fit(X_train,y_train)
s.predict